In [ ]:
TFLITE_FILENAME = 'efficientdet-lite-band.tflite'
LABELS_FILENAME = 'band-labels.txt'

In [ ]:
! python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, scale_factor, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    draw.rectangle([(bbox.xmin * scale_factor, bbox.ymin * scale_factor),
                    (bbox.xmax * scale_factor, bbox.ymax * scale_factor)],
                   outline=color, width=3)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=15)
    draw.text((bbox.xmin * scale_factor + 4, bbox.ymin * scale_factor + 4),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

# Load the TF Lite model
labels = read_label_file(LABELS_FILENAME)
interpreter = tflite.Interpreter(TFLITE_FILENAME)
interpreter.allocate_tensors()

# Resize the image for input
image = Image.open('Resistor_+0.jpg')
_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference
interpreter.invoke()
objs = detect.get_objects(interpreter, score_threshold=0.4, image_scale=scale)

# Resize again to a reasonable size for display
display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
image = image.resize((display_width, int(display_width * height_ratio)))
draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)
image

In [ ]:

image2 = Image.open(img_path)


#george u can come back and add the padding here to the bbox
bands = []
for i in objs:
  img_crop = image2.crop(box=i.bbox)
  bands.append(img_crop)

img_crop



In [ ]:
!pip install opencv-python-headless==4.1.2.30

In [1]:
import numpy as np
import cv2
open_cv_bands=[]
for i in bands:
  open_cv_image = np.array(i) 
  # Convert RGB to BGR 
  open_cv_bands.append(open_cv_image[:, :, ::-1].copy())

#ur script is below

#"Black","Brown","Grey","White"
colors = ["Black","Brown","Red","Orange","Yellow","Green","Blue","Violet","Gold"]
values = [0,1,2,3,4,5,6,7,-1]
all_lower = np.array([[0,22,13],[5,40,48],[0,100,100],[10,150,60],[30,90,45],[40,110,40],[94,80,2],[165,150,65],[13,125,95]])
all_higher = np.array([[50,255,40],[23,255,90],[10,255,255],[20,180,100],[50,255,255],[60,150,65],[126,255,255],[180,255,255],[18,150,150]])

final_colors=[]
for i in open_cv_bands:
  band_image = np.array(i)
  #result=image.copy()
  #cv2.imshow("Original", image)
    
  hsv = cv2.cvtColor(band_image, cv2.COLOR_BGR2HSV)
  
  max=0
  save_i=0
  for i in range(len(colors)):
      # lower boundary RED color range values; Hue (0 - 10)
      lower = all_lower[i]
      upper = all_higher[i]

      
      mask = cv2.inRange(hsv, lower, upper)
      if(i==0):
          save_mask=mask.copy()
      #result = cv2.bitwise_and(image, image, mask=mask)
      if(np.sum(mask)>max):
          max=np.sum(mask)
          save_i=i
          save_mask=mask.copy()




  final_colors.append(colors[save_i])

  #result = cv2.bitwise_and(result, result, mask=save_mask)
  # cv2.imshow('mask', save_mask)
  # cv2.imshow('result',result)

  # print(max)
  # cv2.waitKey(0)
  # cv2.destroyAllWindows()



NameError: ignored

In [ ]:
final_colors.reverse()
def color2res(bands,colors,values):
    flag=0
    if(bands[0]=="Gold"):
      bands.reverse()
      flag=1  
    if(len(bands)==4):
        resistance =  (values[colors.index(bands[0])]*10 + values[colors.index(bands[1])]) * pow(10,(values[colors.index(bands[2])]))
    else:
        resistance =  (values[colors.index(bands[0])]*100 + values[colors.index(bands[1])]*10+values[colors.index(bands[2])]) * pow(10,(values[colors.index(bands[2])]))

    if flag==1:
      bands.reverse()
    return resistance
  
resistance = color2res(final_colors,colors,values)

In [ ]:
import matplotlib.pyplot as plt

display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
#image = image.resize((display_width, int(display_width * height_ratio)))
#draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)

#draw = ImageDraw.Draw(image) 
  
  
text = str(resistance)
  
# drawing text size
#draw.text((5, 5), text, align ="left") 

print(final_colors)
image_original = Image.open(img_path)

f = plt.figure()
f.add_subplot(1,2, 1)
plt.imshow(np.rot90(image_original,2))
f.add_subplot(1,2, 2)
plt.imshow(np.rot90(image,2))
plt.show(block=True)


